In [1]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras import backend as K
import os
import numpy as np
from collections import Counter
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
pet_images_fn = [fn for fn in os.listdir('pet_images') if fn.endswith('.jpg')]
labels = []
idx_to_labels = []
label_to_idx = {}
for fn in pet_images_fn:
    label, _ = fn.rsplit('_', 1)
    if not label in label_to_idx:
        label_to_idx[label] = len(idx_to_labels)
        idx_to_labels.append(label)
    labels.append(label_to_idx[label])
len(idx_to_labels)

37

In [3]:
def fetch_pet(pet):
    img = image.load_img('pet_images/' + pet, target_size=(299, 299))
    return image.img_to_array(img)

img_vector = np.asarray([fetch_pet(pet) for pet in pet_images_fn])

In [4]:
base_model = InceptionV3(weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False

pool_2d = GlobalAveragePooling2D(name='pool_2d')(base_model.output)
dense = Dense(1024, name='dense', activation='relu')(pool_2d)
predictions = Dense(len(idx_to_labels), activation='softmax')(dense)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 0.9448
y = np.zeros((len(labels), len(idx_to_labels)))
for idx, label in enumerate(labels):
    y[idx][label] = 1

model.fit(
    img_vector, y,
    batch_size=128,
    epochs=15,
    verbose=2
)

Epoch 1/15


In [34]:
unfreeze = False
for layer in base_model.layers:
    if unfreeze:
        layer.trainable = True
    if layer.name == 'mixed9':
        unfreeze = True
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
model.fit(
    img_vector, y,
    batch_size=128,
    epochs=15,
    verbose=2
)

Epoch 1/15
35s - loss: 0.2204 - acc: 0.9530
Epoch 2/15
34s - loss: 0.0913 - acc: 0.9754
Epoch 3/15
35s - loss: 0.0484 - acc: 0.9859
Epoch 4/15
35s - loss: 0.0369 - acc: 0.9889
Epoch 5/15
35s - loss: 0.0331 - acc: 0.9911
Epoch 6/15
35s - loss: 0.0275 - acc: 0.9919
Epoch 7/15
35s - loss: 0.0238 - acc: 0.9938
Epoch 8/15
35s - loss: 0.0256 - acc: 0.9934
Epoch 9/15
35s - loss: 0.0213 - acc: 0.9957
Epoch 10/15
35s - loss: 0.0193 - acc: 0.9958
Epoch 11/15
35s - loss: 0.0185 - acc: 0.9961
Epoch 12/15
35s - loss: 0.0190 - acc: 0.9962
Epoch 13/15
35s - loss: 0.0194 - acc: 0.9961
Epoch 14/15
35s - loss: 0.0175 - acc: 0.9963
Epoch 15/15
35s - loss: 0.0172 - acc: 0.9963


In [ ]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
[(np.argsort(n)[-1], n[np.argsort(n)[-1]]) for n in p]